<a href="https://colab.research.google.com/github/AyushGupta30/25MCS0043_CAO_LAB/blob/master/_GuardianX_Al_(DeBERTa)final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# GuardianX – DeBERTa High-Accuracy Notebook (NYC 311)

This notebook supports:
- Full NYC 311 CSV loading
- Long sentence intent recognition
- Combined Complaint Type + Descriptor
- Auto-balanced classes
- DeBERTa model with high accuracy


In [ ]:
!pip install -U transformers datasets accelerate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import re
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


In [ ]:

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)   # remove symbols
    text = re.sub(r"\s+", " ", text).strip()     # clean spaces
    return text



In [ ]:

KEYWORD_TO_CLASS = {
    "fire": "Emergency",
    "accident": "Emergency",
    "emergency": "Emergency",
    "ambulance": "Emergency",

    "traffic": "Traffic",
    "jam": "Traffic",
    "signal": "Traffic",
    "road": "Traffic",

    "garbage": "Garbage",
    "waste": "Garbage",
    "trash": "Garbage",
    "dustbin": "Garbage",

    "water": "Water",
    "leak": "Water",
    "pipe": "Water",
    "tap": "Water",
    "supply": "Water",

    "electricity": "Electricity",
    "electric": "Electricity",
    "power": "Electricity",
    "light": "Electricity"
}

def assign_label(text):
    text = preprocess_text(text)
    for k, v in KEYWORD_TO_CLASS.items():
        if k in text:
            return v
    return None


In [ ]:

CSV_PATH = "/content/drive/MyDrive/311_Service_Requests_from_2011.csv"
df = pd.read_csv(CSV_PATH, low_memory=False)


In [ ]:
import unicodedata

def normalize_col(c):
    c = unicodedata.normalize("NFKD", c)   # remove unicode weirdness
    c = c.encode("ascii", "ignore").decode("ascii")
    c = c.strip().lower()
    c = " ".join(c.split())
    return c

df.columns = [normalize_col(c) for c in df.columns]

print("Normalized columns:")
for i, c in enumerate(df.columns):
    print(i, repr(c))


Normalized columns:
0 'unique key'
1 'created date'
2 'closed date'
3 'agency'
4 'agency name'
5 'complaint type'
6 'descriptor'
7 'location type'
8 'incident zip'
9 'incident address'
10 'street name'
11 'cross street 1'
12 'cross street 2'
13 'intersection street 1'
14 'intersection street 2'
15 'address type'
16 'city'
17 'landmark'
18 'facility type'
19 'status'
20 'due date'
21 'resolution description'
22 'resolution action updated date'
23 'community board'
24 'borough'
25 'x coordinate (state plane)'
26 'y coordinate (state plane)'
27 'park facility name'
28 'park borough'
29 'school name'
30 'school number'
31 'school region'
32 'school code'
33 'school phone number'
34 'school address'
35 'school city'
36 'school state'
37 'school zip'
38 'school not found'
39 'school or citywide complaint'
40 'vehicle type'
41 'taxi company borough'
42 'taxi pick up location'
43 'bridge highway name'
44 'bridge highway direction'
45 'road ramp'
46 'bridge highway segment'
47 'garage lot name'

In [ ]:
def find_col(keyword_list):
    for col in df.columns:
        for kw in keyword_list:
            if kw in col:
                return col
    return None

agency_col = find_col(["agency"])
complaint_col = find_col(["complaint"])
descriptor_col = find_col(["descriptor", "description", "desc"])

print("Detected columns:")
print("Agency:", agency_col)
print("Complaint:", complaint_col)
print("Descriptor:", descriptor_col)

text_cols = [c for c in [agency_col, complaint_col, descriptor_col] if c is not None]

if not text_cols:
    raise ValueError("No valid text columns found even after auto-detection")

df["text"] = df[text_cols].astype(str).agg(" ".join, axis=1)
df = df[["text"]].dropna()

print("Using columns:", text_cols)
print("After combine:", df.shape)


Detected columns:
Agency: agency
Complaint: complaint type
Descriptor: descriptor
Using columns: ['agency', 'complaint type', 'descriptor']
After combine: (1917212, 1)


In [ ]:

df["label"] = df["text"].apply(assign_label)
df = df.dropna(subset=["label"])
df["text"] = df["text"].apply(preprocess_text)
df = df[["text", "label"]]
print(df["label"].value_counts())


label
Electricity    144457
Water          134717
Traffic         59654
Emergency       18039
Garbage          7960
Name: count, dtype: int64


In [ ]:

min_size = df["label"].value_counts().min()
balanced = []

for label in df["label"].unique():
    subset = df[df["label"] == label]
    balanced.append(resample(subset, n_samples=min_size, random_state=42))

df = pd.concat(balanced).sample(frac=1).reset_index(drop=True)


In [ ]:

label_encoder = LabelEncoder()
df["labels"] = label_encoder.fit_transform(df["label"])

num_labels = len(label_encoder.classes_)


In [ ]:

MODEL_NAME = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

MAX_LEN = 128

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )


In [ ]:

from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

train_ds = Dataset.from_pandas(train_df).map(tokenize, batched=True)
test_ds = Dataset.from_pandas(test_df).map(tokenize, batched=True)

train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

test_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Map:   0%|          | 0/31840 [00:00<?, ? examples/s]

Map:   0%|          | 0/7960 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./guardianx_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs"
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-2293869835.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss
1,0.000000,0.001167
2,0.000000,0.000001
3,0.000000,0.000000


TrainOutput(global_step=11940, training_loss=0.010161809034773673, metrics={'train_runtime': 1897.913, 'train_samples_per_second': 50.329, 'train_steps_per_second': 6.291, 'total_flos': 3163603399925760.0, 'train_loss': 0.010161809034773673, 'epoch': 3.0})

In [ ]:

def predict_department(text):
    text = preprocess_text(text)
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

    with torch.no_grad():
        outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=1)
    confidence, pred = torch.max(probs, dim=1)
    label = label_encoder.inverse_transform(pred.cpu().numpy())[0]

    if confidence.item() < 0.55:
        return "General Support", confidence.item()

    return label, confidence.item()


In [ ]:
MODEL_SAVE_PATH = "/content/drive/MyDrive/GuardianX_DeBERTa_Final"


In [ ]:
# Save model & tokenizer
trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print("Model and tokenizer saved to:", MODEL_SAVE_PATH)


Model and tokenizer saved to: /content/drive/MyDrive/GuardianX_DeBERTa_Final


In [ ]:
import joblib

joblib.dump(label_encoder, f"{MODEL_SAVE_PATH}/label_encoder.pkl")

print("Label encoder saved")


Label encoder saved


In [ ]:
import os

os.listdir(MODEL_SAVE_PATH)


['config.json',
 'model.safetensors',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'added_tokens.json',
 'spm.model',
 'tokenizer.json',
 'training_args.bin',
 'label_encoder.pkl']

*start* from here

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib
import torch
import torch.nn.functional as F

MODEL_SAVE_PATH = "/content/drive/MyDrive/GuardianX_DeBERTa_Final"

tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)
label_encoder = joblib.load(f"{MODEL_SAVE_PATH}/label_encoder.pkl")

model.eval()
print("Model loaded successfully")


Loading weights:   0%|          | 0/106 [00:00<?, ?it/s]

Model loaded successfully


In [ ]:
def predict_with_fallback(text, threshold=0.55):
    text = preprocess_text(text)

    # ✅ First: keyword match
    for k, v in KEYWORD_TO_CLASS.items():
        if k in text:
            return v, 1.0

    # Then use model
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=1)
    confidence, pred = torch.max(probs, dim=1)

    label = label_encoder.inverse_transform(pred.cpu().numpy())[0]

    return label, confidence.item()



In [ ]:
# ============================================
# REQUIRED DEFINITIONS FOR GRADIO INTERFACE
# ============================================

import re

# Text preprocessing
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Keyword mapping
KEYWORD_TO_CLASS = {
    # Emergency
    "fire": "Emergency",
    "accident": "Emergency",
    "ambulance": "Emergency",

    # Traffic
    "traffic": "Traffic",
    "jam": "Traffic",
    "signal": "Traffic",
    "road": "Traffic",

    # Garbage
    "garbage": "Garbage",
    "waste": "Garbage",
    "trash": "Garbage",
    "dustbin": "Garbage",

    # Water
    "water": "Water",
    "leak": "Water",
    "pipe": "Water",
    "tap": "Water",
    "supply": "Water",

    # Electricity  ✅
    "electric": "Electricity",
    "electricity": "Electricity",
    "power": "Electricity",
    "light": "Electricity",
    "current": "Electricity",
    "voltage": "Electricity"
}


# SLA Table (Service Level Agreement times)
SLA_TABLE = {
    "Emergency": "00:30",    # 30 minutes
    "Water": "02:00",        # 2 hours
    "Electricity": "04:00",  # 4 hours
    "Traffic": "01:00",      # 1 hour
    "Garbage": "24:00"       # 24 hours
}

print("✓ All required definitions loaded")

✓ All required definitions loaded


In [ ]:
!pip install -q gradio joblib


In [ ]:
# ==================== CELL 3: Import All Libraries ====================
import pandas as pd
import numpy as np
import re
import torch
import torch.nn.functional as F
import gradio as gr
import sqlite3
import threading
import time
import datetime
import random
import smtplib
import requests
import matplotlib.pyplot as plt
from email.mime.text import MIMEText
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [ ]:
# ==================== CELL 4: Load Trained Model ====================
MODEL_SAVE_PATH = "/content/drive/MyDrive/GuardianX_DeBERTa_Final"

tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)
label_encoder = joblib.load(f"{MODEL_SAVE_PATH}/label_encoder.pkl")

model.eval()
print("Model loaded successfully")


Loading weights:   0%|          | 0/106 [00:00<?, ?it/s]

Model loaded successfully


In [ ]:
# ==================== CELL 5: Configuration ====================
KEYWORD_TO_CLASS = {
    "fire": "Emergency", "accident": "Emergency", "emergency": "Emergency", "ambulance": "Emergency",
    "traffic": "Traffic", "jam": "Traffic", "signal": "Traffic", "road": "Traffic",
    "garbage": "Garbage", "waste": "Garbage", "trash": "Garbage", "dustbin": "Garbage",
    "water": "Water", "leak": "Water", "pipe": "Water", "tap": "Water", "supply": "Water",
    "electricity": "Electricity", "electric": "Electricity", "power": "Electricity", "light": "Electricity"
}

ACTIONS = {
    "Emergency": ["Fire Agent", "Medical Agent"],
    "Traffic": ["Traffic Control", "Road Safety"],
    "Garbage": ["Sanitation Team", "Waste Management"],
    "Water": ["Water Supply", "Plumbing Team"],
    "Electricity": ["Power Grid", "Line Maintenance"]
}

AGENT_DECISIONS = {
    "Emergency": ["Alert control room", "Dispatch fire unit", "Dispatch medical unit", "Resolve"],
    "Traffic": ["Assess traffic flow", "Deploy traffic officers", "Update signals", "Resolve"],
    "Garbage": ["Schedule pickup", "Dispatch sanitation team", "Clear area", "Resolve"],
    "Water": ["Check water supply", "Dispatch repair team", "Fix issue", "Resolve"],
    "Electricity": ["Locate outage", "Dispatch technicians", "Restore power", "Resolve"]
}

SLA_TABLE = {
    "Emergency": "00:30", "Traffic": "02:00", "Garbage": "24:00",
    "Water": "12:00", "Electricity": "08:00"
}

EMAIL_FROM = "iamayushgupta3011@gmail.com"
EMAIL_PASSWORD = "wuzycpvaphwhypdw"
#EMAIL_TO = "ayush30nov2002gupta@gmail.com"
API_URL = "https://webhook.site/your-unique-url"
CREATOR_NAME = ""

print("Configuration loaded")

Configuration loaded


In [ ]:
# ==================== CELL 6: Database Setup ====================
def view_db():
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()
        cur.execute("""
            SELECT ticket_no, created_by, department, agent, complaint,
                   location, priority, status, created_at, closed_at
            FROM tickets ORDER BY created_at DESC
        """)
        rows = cur.fetchall()
        conn.close()

        if len(rows) == 0:
            return pd.DataFrame(columns=[
                "Ticket No", "Created By", "Department", "Agent",
                "Complaint", "Location", "Priority", "Status",
                "Created At", "Closed At"
            ])

        df = pd.DataFrame(rows, columns=[
            "Ticket No", "Created By", "Department", "Agent",
            "Complaint", "Location", "Priority", "Status",
            "Created At", "Closed At"
        ])
        df["Complaint"] = df["Complaint"].apply(lambda x: x[:40] + "..." if len(str(x)) > 40 else x)
        return df
    except Exception as e:
        print(f"❌ Database view error: {e}")
        return pd.DataFrame()

def get_ticket_stats():
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()

        cur.execute("SELECT COUNT(*) FROM tickets")
        total = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE status='OPEN'")
        open_count = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE status='CLOSED'")
        closed_count = cur.fetchone()[0]

        conn.close()

        return f"📊 Total: {total} | 🟢 Open: {open_count} | ✅ Closed: {closed_count}"
    except Exception as e:
        return f"Stats unavailable: {str(e)}"



In [ ]:
# ==================== CELL 7: Utility Functions ====================
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def predict_with_fallback(text, threshold=0.55):
    text = preprocess_text(text)
    for k, v in KEYWORD_TO_CLASS.items():
        if k in text:
            return v, 1.0

    inputs = tokenizer(text, return_tensors="pt", truncation=True,
                      padding="max_length", max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=1)
    confidence, pred = torch.max(probs, dim=1)

    if confidence.item() < threshold:
        return "General Support", confidence.item()

    label = label_encoder.inverse_transform(pred.cpu().numpy())[0]
    return label, confidence.item()

def detect_priority(text):
    text = text.lower()
    if any(word in text for word in ["emergency", "urgent", "fire", "accident", "critical"]):
        return "🔴 Critical"
    elif any(word in text for word in ["important", "soon", "asap"]):
        return "🟠 High"
    elif any(word in text for word in ["moderate", "medium"]):
        return "🟡 Medium"
    return "🟢 Low"

def send_email(subject, body, recipient_email):
    """Modified to accept recipient email as parameter"""
    try:
        msg = MIMEText(body)
        msg["Subject"] = subject
        msg["From"] = EMAIL_FROM
        msg["To"] = recipient_email  # Use the provided email

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_FROM, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"📧 Email sent to: {recipient_email}")
        return True
    except Exception as e:
        print(f"❌ Email error: {e}")
        return False

def call_api(agent, text, location):
    try:
        payload = {
            "agent": agent,
            "complaint": text,
            "location": location,
            "timestamp": datetime.datetime.now().isoformat()
        }
        response = requests.post(API_URL, json=payload, timeout=5)
        print(f"🌐 API called: {response.status_code}")
    except Exception as e:
        print(f"❌ API error: {e}")

In [ ]:
# ==================== CELL 8: Global State ====================
data_store = []
history = []
ticket_status = {}

def add_to_history(message):
    timestamp = datetime.datetime.now().strftime("%H:%M:%S")
    history.append(f"[{timestamp}] {message}")
    if len(history) > 25:
        history.pop(0)

def get_history_text():
    return "\n".join(history)

def get_ticket_followup(ticket_no):
    return ticket_status.get(ticket_no, "No active ticket")

In [ ]:
def agent_job(agent, text, location, priority, creator_name, user_email):
    """Create ticket and save to database - NO AUTO-CLOSE"""
    ticket_no = f"TKT-{random.randint(100000, 999999)}"
    department = agent

    try:
        add_to_history("")
        add_to_history("="*60)
        add_to_history(f"🎫 CREATING TICKET IN DATABASE")
        add_to_history("="*60)
        add_to_history(f"🎫 Ticket Number: {ticket_no}")
        add_to_history(f"👤 Created by: {creator_name}")
        add_to_history(f"📧 Email: {user_email}")
        add_to_history(f"🏢 Department: {department}")
        add_to_history(f"🤖 Agent: {agent}")
        add_to_history(f"📍 Location: {location}")
        add_to_history(f"⚠️ Priority: {priority}")
        add_to_history(f"📊 Status: OPEN")

        # Save to database
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()
        created_time = datetime.datetime.now().isoformat()

        cur.execute("""
            INSERT INTO tickets
            (ticket_no, created_by, email, department, agent, complaint, location, priority, status, created_at)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, 'OPEN', ?)
        """, (ticket_no, creator_name, user_email, department, agent, text, location, priority, created_time))

        conn.commit()
        conn.close()

        add_to_history(f"✅ Ticket {ticket_no} saved to database successfully!")
        add_to_history(f"📊 Database status: OPEN")

        # Update data store for analytics
        data_store.append({
            "ticket_no": ticket_no,
            "agent": agent,
            "priority": priority,
            "location": location,
            "date": datetime.date.today().isoformat(),
            "status": "OPEN"
        })

        # Send opening email
        add_to_history(f"📧 Sending opening email to: {user_email}")
        email_sent = send_email(
            f"🆕 New {department} Complaint - {ticket_no}",
            f"""NEW TICKET CREATED

Ticket Number: {ticket_no}
Created By: {creator_name}
Department: {department}
Agent: {agent}
Complaint: {text}
Location: {location}
Priority: {priority}
Status: OPEN
Created At: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Your ticket has been assigned to the {department} Department.
You will receive another email when this ticket is resolved.

Please keep this ticket number for your reference: {ticket_no}

Thank you for using GuardianX AI!
""",
            user_email
        )

        if email_sent:
            add_to_history(f"✅ Opening email sent successfully to {user_email}")
        else:
            add_to_history(f"⚠️ Email sending failed (check email configuration)")

        # Call external API
        call_api(agent, text, location)
        add_to_history(f"🌐 External API notified")

        add_to_history(f"📌 Ticket {ticket_no} is now OPEN - awaiting {department} Department resolution")
        add_to_history("="*60)

        # Update ticket status in memory
        ticket_status[ticket_no] = f"⏳ {agent} Department - Ticket OPEN, awaiting resolution"

        return ticket_no

    except Exception as e:
        print(f"❌ Agent job error: {e}")
        import traceback
        traceback.print_exc()
        add_to_history(f"❌ Error creating ticket: {str(e)}")
        return None

def run_real_agent(agent, text, location, priority, creator_name, user_email):
    """Run agent in thread"""
    ticket_container = [None]

    def thread_wrapper():
        ticket_no = agent_job(agent, text, location, priority, creator_name, user_email)
        ticket_container[0] = ticket_no

    thread = threading.Thread(target=thread_wrapper, daemon=True)
    thread.start()
    thread.join(timeout=3)  # Wait up to 3 seconds for completion

    return ticket_container[0]

In [ ]:
# ==================== CELL 11: Main Run Agent ====================
current_ticket = None

def run_agent(text, location, creator_name=None, user_email=None):
    """Main function that processes complaints and creates tickets"""
    global current_ticket

    try:
        if not text or not location:
            return "", "", "", "", "", get_history_text()

        if not creator_name or creator_name.strip() == "":
            creator_name = "Anonymous"

        # Validate email
        if not user_email or user_email.strip() == "":
            add_to_history("❌ Error: Email is required")
            return "Error: Email is required", "", "", "", "", get_history_text()

        # Get AI prediction for department
        agent, confidence = predict_with_fallback(text)
        agent = agent.title()
        priority = detect_priority(text)

        sub_agents = ", ".join(ACTIONS.get(agent, ["General Support"]))
        decisions = AGENT_DECISIONS.get(agent, ["Analyze", "Assign", "Resolve"])
        decision_text = "\n".join([f"{i+1}. {d}" for i, d in enumerate(decisions)])
        sla = SLA_TABLE.get(agent, "24:00")

        add_to_history("")
        add_to_history("="*60)
        add_to_history(f"🔍 NEW COMPLAINT ANALYSIS STARTED")
        add_to_history("="*60)
        add_to_history(f"📝 Complaint: '{text[:50]}{'...' if len(text) > 50 else ''}'")
        add_to_history(f"📍 Location: {location}")
        add_to_history(f"👤 Submitted by: {creator_name}")
        add_to_history(f"📧 Email: {user_email}")
        add_to_history(f"🤖 AI Analysis: Category identified as {agent} (confidence: {confidence:.2%})")
        add_to_history(f"⏱️ SLA: {sla}")
        add_to_history("")

        # Create ticket through agent system
        ticket_no = run_real_agent(agent, text, location, priority, creator_name, user_email)
        current_ticket = ticket_no

        if ticket_no:
            add_to_history(f"✅ Ticket {ticket_no} successfully created and saved")
        else:
            add_to_history("❌ Error creating ticket")

        return agent, sub_agents, sla, priority, decision_text, get_history_text()

    except Exception as e:
        print(f"🔥 RUN_AGENT ERROR: {e}")
        import traceback
        traceback.print_exc()
        error_msg = f"Error: {str(e)}"
        add_to_history(f"❌ Critical Error: {str(e)}")
        return error_msg, error_msg, error_msg, error_msg, error_msg, get_history_text()

In [ ]:
# ==================== CELL 12: Analytics Functions ====================
def generate_charts():
    if not data_store:
        return None, None, None

    try:
        df = pd.DataFrame(data_store)

        plt.figure(figsize=(8, 6))
        df["agent"].value_counts().plot(kind="bar", color='orange')
        plt.title("Complaints by Agent", fontsize=14, fontweight='bold')
        plt.xlabel("Agent")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig("agent.png")
        plt.clf()

        plt.figure(figsize=(8, 6))
        df["date"].value_counts().sort_index().plot(kind="line", marker='o', color='orange', linewidth=2)
        plt.title("Complaints by Day", fontsize=14, fontweight='bold')
        plt.xlabel("Date")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig("day.png")
        plt.clf()

        plt.figure(figsize=(8, 6))
        df["priority"].value_counts().plot(kind="pie", autopct="%1.1f%%",
                                          colors=['red', 'orange', 'yellow', 'green'],
                                          startangle=90)
        plt.title("Complaints by Priority", fontsize=14, fontweight='bold')
        plt.ylabel("")
        plt.tight_layout()
        plt.savefig("priority.png")
        plt.clf()

        return "agent.png", "day.png", "priority.png"
    except Exception as e:
        print(f"❌ Chart generation error: {e}")
        return None, None, None

def get_ticket_stats():
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()

        cur.execute("SELECT COUNT(*) FROM tickets")
        total = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE status='OPEN'")
        open_count = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE status='CLOSED'")
        closed_count = cur.fetchone()[0]

        conn.close()

        return f"📊 Total: {total} | 🟢 Open: {open_count} | ✅ Closed: {closed_count}"
    except Exception as e:
        return f"Stats unavailable: {str(e)}"

def export_admin():
    try:
        conn = sqlite3.connect("complaints.db")
        df = pd.read_sql_query("SELECT * FROM tickets ORDER BY created_at DESC", conn)
        conn.close()

        if len(df) == 0:
            print("⚠️ No tickets to export")
            return None

        filename = f"guardianx_tickets_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        print(f"✅ Exported {len(df)} tickets to {filename}")
        return filename
    except Exception as e:
        print(f"❌ Export error: {e}")
        return None


In [ ]:
# ==================== DATABASE INITIALIZATION ====================
def init_db():
    """Initialize database with email column"""
    conn = sqlite3.connect("complaints.db")
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS tickets")
    cur.execute("""
        CREATE TABLE tickets (
            ticket_no TEXT PRIMARY KEY,
            created_by TEXT NOT NULL,
            email TEXT NOT NULL,
            department TEXT NOT NULL,
            agent TEXT NOT NULL,
            complaint TEXT NOT NULL,
            location TEXT NOT NULL,
            priority TEXT,
            status TEXT DEFAULT 'OPEN',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            closed_at TIMESTAMP,
            resolution TEXT,
            resolved_by TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Database initialized with email column")

In [ ]:
# ==================== COMPLETE GUARDIANX GRADIO INTERFACE ====================
# Add this to your existing code after all the function definitions

import gradio as gr
import pandas as pd
import sqlite3
import datetime


# ==================== DEPARTMENT FUNCTIONS ====================
def get_department_tickets(department):
    """Get OPEN tickets for a specific department"""
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()
        cur.execute("""
            SELECT ticket_no, created_by, agent, complaint, location,
                   priority, status, created_at
            FROM tickets
            WHERE department = ? AND status = 'OPEN'
            ORDER BY created_at DESC
        """, (department,))
        rows = cur.fetchall()
        conn.close()

        print(f"📊 Found {len(rows)} OPEN tickets for {department} department")

        if len(rows) == 0:
            return pd.DataFrame(columns=[
                "Ticket No", "Created By", "Agent", "Complaint",
                "Location", "Priority", "Status", "Created At"
            ])

        df = pd.DataFrame(rows, columns=[
            "Ticket No", "Created By", "Agent", "Complaint",
            "Location", "Priority", "Status", "Created At"
        ])
        df["Complaint"] = df["Complaint"].apply(lambda x: str(x)[:40] + "..." if len(str(x)) > 40 else str(x))
        return df
    except Exception as e:
        print(f"❌ Error fetching department tickets: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

def get_department_closed_tickets(department):
    """Get CLOSED tickets for a specific department"""
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()
        cur.execute("""
            SELECT ticket_no, created_by, agent, complaint, location,
                   priority, status, created_at, resolution, closed_at, resolved_by
            FROM tickets
            WHERE department = ? AND status = 'CLOSED'
            ORDER BY closed_at DESC
        """, (department,))
        rows = cur.fetchall()
        conn.close()

        print(f"📊 Found {len(rows)} CLOSED tickets for {department} department")

        if len(rows) == 0:
            return pd.DataFrame(columns=[
                "Ticket No", "Created By", "Agent", "Complaint",
                "Location", "Priority", "Status", "Created At",
                "Resolution", "Closed At", "Resolved By"
            ])

        df = pd.DataFrame(rows, columns=[
            "Ticket No", "Created By", "Agent", "Complaint",
            "Location", "Priority", "Status", "Created At",
            "Resolution", "Closed At", "Resolved By"
        ])
        df["Complaint"] = df["Complaint"].apply(lambda x: str(x)[:30] + "..." if len(str(x)) > 30 else str(x))
        df["Resolution"] = df["Resolution"].apply(lambda x: str(x)[:40] + "..." if x and len(str(x)) > 40 else str(x))
        return df
    except Exception as e:
        print(f"❌ Error fetching closed tickets: {e}")
        return pd.DataFrame()

def get_department_stats(department):
    """Get statistics for a specific department"""
    try:
        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()

        cur.execute("SELECT COUNT(*) FROM tickets WHERE department = ?", (department,))
        total = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE department = ? AND status = 'OPEN'", (department,))
        open_count = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM tickets WHERE department = ? AND status = 'CLOSED'", (department,))
        closed_count = cur.fetchone()[0]

        conn.close()

        print(f"📊 {department} stats - Total: {total}, Open: {open_count}, Closed: {closed_count}")

        return f"📊 Total: {total} | 🟢 Open: {open_count} | ✅ Closed: {closed_count}"
    except Exception as e:
        print(f"❌ Error getting stats: {e}")
        return f"Stats unavailable: {str(e)}"

def mark_ticket_resolved(ticket_no, resolution_note, department):
    """Mark ticket as resolved - ONLY way to close tickets"""
    try:
        if not ticket_no or not ticket_no.strip():
            return "❌ Error: Please enter a ticket number", get_history_text()

        if not resolution_note or not resolution_note.strip():
            return "❌ Error: Please enter resolution details", get_history_text()

        conn = sqlite3.connect("complaints.db")
        cur = conn.cursor()

        # Get ticket details
        cur.execute("""
            SELECT created_by, complaint, location, department, email
            FROM tickets
            WHERE ticket_no = ? AND status = 'OPEN'
        """, (ticket_no,))
        ticket_data = cur.fetchone()

        if not ticket_data:
            conn.close()
            return f"❌ Error: Ticket {ticket_no} not found or already closed", get_history_text()

        created_by, complaint, location, ticket_dept, email = ticket_data

        # Check department match
        if ticket_dept != department:
            conn.close()
            return f"❌ Error: This ticket belongs to {ticket_dept} department, not {department}", get_history_text()

        closed_time = datetime.datetime.now().isoformat()
        resolved_by_text = f"{department} Department"

        # Update ticket
        cur.execute("""
            UPDATE tickets
            SET status = 'CLOSED',
                resolution = ?,
                closed_at = ?,
                resolved_by = ?
            WHERE ticket_no = ?
        """, (resolution_note, closed_time, resolved_by_text, ticket_no))

        conn.commit()
        conn.close()

        # Add to history
        add_to_history("")
        add_to_history("="*60)
        add_to_history(f"✅ TICKET MANUALLY RESOLVED BY DEPARTMENT")
        add_to_history("="*60)
        add_to_history(f"🎫 Ticket: {ticket_no}")
        add_to_history(f"🏢 Department: {department}")
        add_to_history(f"👤 Resolved by: {resolved_by_text}")
        add_to_history(f"📝 Resolution: {resolution_note}")
        add_to_history(f"📊 Status updated: OPEN → CLOSED")
        add_to_history(f"⏰ Closed at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        # Update memory
        ticket_status[ticket_no] = f"✅ Resolved by {department} Department"

        # Update data store
        for item in data_store:
            if item.get("ticket_no") == ticket_no:
                item["status"] = "CLOSED"
                break

        # Send closure email
        add_to_history(f"📧 Sending closure email to: {email}")
        email_sent = send_email(
            f"✅ Ticket {ticket_no} Resolved - {department}",
            f"""TICKET RESOLUTION NOTIFICATION

Dear {created_by},

Your complaint has been successfully resolved by the {department} Department!

Ticket Number: {ticket_no}
Department: {department}
Complaint: {complaint}
Location: {location}

Status: CLOSED
Resolved By: {resolved_by_text}
Resolution: {resolution_note}
Closed At: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Thank you for using GuardianX AI!

Best regards,
GuardianX Team
""",
            email
        )

        if email_sent:
            add_to_history(f"✅ Closure email sent successfully to {email}")
        else:
            add_to_history(f"⚠️ Email sending failed")

        add_to_history("="*60)

        return f"✅ SUCCESS!\n\nTicket {ticket_no} marked as RESOLVED\n📧 Closure email sent to: {email}\n📊 Database updated: OPEN → CLOSED\n\nYou can now close this form.", get_history_text()

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        print(f"Resolution error: {e}")
        import traceback
        traceback.print_exc()
        add_to_history(f"❌ Error resolving ticket {ticket_no}: {str(e)}")
        return error_msg, get_history_text()

def refresh_department_view(department):
    """Refresh department view"""
    stats = get_department_stats(department)
    open_tickets = get_department_tickets(department)
    closed_tickets = get_department_closed_tickets(department)
    return stats, open_tickets, closed_tickets, get_history_text()

# ==================== MAIN GRADIO INTERFACE ====================
init_db()

custom_css = """
body {
    background:#070707;
    color:#f5f5f5;
    font-family:'Segoe UI', Arial, sans-serif;
}

.header-area{
    padding:30px;
    background:linear-gradient(135deg,#ff8a00,#e65100);
    border-radius:20px;
    margin-bottom:25px;
    box-shadow:0 0 40px rgba(255,138,0,.5);
    text-align:center;
}

.card-node{
    background:#111 !important;
    border:2px solid #ff8a00 !important;
    border-radius:20px !important;
    padding:20px !important;
    box-shadow:0 0 20px rgba(255,138,0,.3);
}

textarea, input{
    background:#0d0d0d !important;
    color:white !important;
    border:1px solid #444 !important;
    border-radius:8px !important;
}

.log-box textarea{
    background:black !important;
    color:#3aff87 !important;
    font-family:'Courier New', monospace !important;
    font-size:13px !important;
}

button{
    background:linear-gradient(135deg,#ff8a00,#e65100) !important;
    color:#000 !important;
    font-weight:bold !important;
    border:none !important;
    border-radius:12px !important;
    padding:12px 24px !important;
    transition:all 0.3s !important;
}

button:hover{
    box-shadow:0 0 20px rgba(255,138,0,.6) !important;
    transform:translateY(-2px) !important;
}
"""

app = gr.Blocks(css=custom_css)

with app:
    gr.HTML("""
    <div class='header-area'>
        <h1 style='margin:0; font-size:2.5em;'><strong>🛡️ GuardianX-AI</strong></h1>
        <p style='margin:10px 0 5px 0; font-size:1.2em;'>Always Watching - Always Protecting</p>
        <p style='margin:0; font-size:1em; opacity:0.9;'>Next-Gen Autonomous Dispatch Portal</p>
    </div>
    """)

    with gr.Tabs():
        # ==================== COMMAND CENTER (CITIZEN) ====================
        with gr.TabItem("🚀 Command Center"):
            with gr.Row(equal_height=True):
                with gr.Column(scale=1):
                    with gr.Group(elem_classes="card-node"):
                        gr.Markdown("## 📝 Citizen Input")
                        creator_inp = gr.Textbox(
                            label="Your Name",
                            placeholder="Enter your name"
                        )
                        email_inp = gr.Textbox(
                            label="Email (for updates)",
                            placeholder="your.email@example.com"
                        )
                        inp = gr.Textbox(
                            lines=8,
                            label="Complaint Description",
                            placeholder="Describe your complaint in detail..."
                        )
                        location = gr.Textbox(
                            label="Location",
                            placeholder="Enter the location...",
                            lines=2
                        )
                        submit = gr.Button("🚀 Submit Complaint", size="lg")

                with gr.Column(scale=1):
                    with gr.Group(elem_classes="card-node"):
                        gr.Markdown("## 🎯 Agent Decision Panel")
                        with gr.Row():
                            agent_out = gr.Textbox(label="Primary Agent", interactive=False, scale=2)
                            sla_out = gr.Textbox(label="SLA", interactive=False, scale=1)
                        priority_out = gr.Textbox(label="Priority Level", interactive=False)
                        subs_out = gr.Textbox(label="Sub Agents", interactive=False, lines=2)
                        actions_out = gr.Textbox(
                            label="Agent Actions",
                            lines=8,
                            interactive=False
                        )

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("## 🧠 System Memory")
                hist = gr.Textbox(
                    lines=12,
                    elem_classes="log-box",
                    label="Activity Log",
                    interactive=False,
                    show_copy_button=True
                )

        # ==================== DEPARTMENT DASHBOARD ====================
        with gr.TabItem("🏢 Department Dashboard"):
            with gr.Group(elem_classes="card-node"):
                gr.Markdown("## 🏢 Department-Specific Dashboard")

                gr.Markdown("### Select Your Department")
                dept_selector = gr.Dropdown(
                    choices=["Emergency", "Water", "Electricity", "Traffic", "Garbage"],
                    value="Emergency",
                    label="Department",
                    interactive=True
                )

                dept_stats = gr.Textbox(
                    label="Department Statistics",
                    interactive=False,
                    value=get_department_stats("Emergency")
                )

                dept_refresh_btn = gr.Button("🔄 Refresh Dashboard", size="lg")

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("### 🟢 Your Department Tickets (OPEN)")
                dept_open_tickets = gr.Dataframe(
                    headers=[
                        "Ticket No", "Created By", "Agent", "Complaint",
                        "Location", "Priority", "Status", "Created At"
                    ],
                    datatype=["str"] * 8,
                    label="Open Tickets - Need Resolution",
                    interactive=False,
                    wrap=True,
                    value=get_department_tickets("Emergency")
                )

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("### ✅ Mark Ticket as Resolved")

                ticket_no_resolve = gr.Textbox(
                    label="Ticket Number",
                    placeholder="Enter ticket number (e.g., TKT-123456)"
                )

                resolution_note = gr.Textbox(
                    label="Resolution Details",
                    placeholder="Enter how the issue was resolved...",
                    lines=4
                )

                resolve_btn = gr.Button("✅ Mark Resolved", size="lg")

                resolution_status = gr.Textbox(
                    label="Resolution Status",
                    interactive=False,
                    lines=4
                )

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("### ✅ Resolved Tickets (CLOSED)")
                dept_closed_tickets = gr.Dataframe(
                    headers=[
                        "Ticket No", "Created By", "Agent", "Complaint",
                        "Location", "Priority", "Status", "Created At",
                        "Resolution", "Closed At", "Resolved By"
                    ],
                    datatype=["str"] * 11,
                    label="Closed Tickets - Successfully Resolved",
                    interactive=False,
                    wrap=True,
                    value=get_department_closed_tickets("Emergency")
                )

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("### 🧠 Department Activity Log")
                dept_hist = gr.Textbox(
                    lines=10,
                    elem_classes="log-box",
                    label="Activity Log",
                    interactive=False,
                    show_copy_button=True
                )

        # ==================== ADMIN PANEL ====================
        with gr.TabItem("🗄️ Admin"):
            with gr.Group(elem_classes="card-node"):
                gr.Markdown("## 📂 Complaint Database Management")
                stats_display = gr.Textbox(
                    label="Database Statistics",
                    interactive=False,
                    value=get_ticket_stats()
                )
                with gr.Row():
                    view_btn = gr.Button("🔄 Refresh Database", size="lg")
                    export_btn = gr.Button("📥 Export All Records", size="lg")
                db_table = gr.Dataframe(
                    headers=[
                        "Ticket No", "Created By", "Department", "Agent",
                        "Complaint", "Location", "Priority", "Status",
                        "Created At", "Closed At"
                    ],
                    datatype=["str"] * 10,
                    label="All Tickets",
                    interactive=False,
                    wrap=True,
                    value=view_db()
                )
                file_out = gr.File(label="Download CSV Export")

            with gr.Group(elem_classes="card-node"):
                gr.Markdown("## 📈 Live Analytics Dashboard")
                chart_btn = gr.Button("🔄 Generate Analytics", size="lg")
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### 📊 By Agent")
                        c1 = gr.Image(height=350)
                    with gr.Column(scale=1):
                        gr.Markdown("### 📅 By Day")
                        c2 = gr.Image(height=350)
                    with gr.Column(scale=1):
                        gr.Markdown("### ⚠️ By Priority")
                        c3 = gr.Image(height=350)

    # ============================================================================
    # EVENT HANDLERS
    # ============================================================================

    def run_agent_wrapper(text, location, creator_name, user_email):
        result = run_agent(text, location, creator_name, user_email)
        if len(result) == 6:
            return result[0], result[1], result[2], result[3], result[4], result[5]
        else:
            return result

    submit.click(
        run_agent_wrapper,
        [inp, location, creator_inp, email_inp],
        [agent_out, subs_out, sla_out, priority_out, actions_out, hist]
    )

    auto_refresh = gr.Timer(value=2, active=True)
    auto_refresh.tick(
        fn=lambda: get_history_text(),
        inputs=None,
        outputs=[hist, dept_hist]
    )

    dept_selector.change(
        refresh_department_view,
        inputs=[dept_selector],
        outputs=[dept_stats, dept_open_tickets, dept_closed_tickets, dept_hist]
    )

    dept_refresh_btn.click(
        refresh_department_view,
        inputs=[dept_selector],
        outputs=[dept_stats, dept_open_tickets, dept_closed_tickets, dept_hist]
    )

    def handle_resolve(ticket_no, resolution_note, department):
        status_msg, history = mark_ticket_resolved(ticket_no, resolution_note, department)
        stats, open_tickets, closed_tickets, updated_history = refresh_department_view(department)
        return status_msg, updated_history, stats, open_tickets, closed_tickets, "", ""

    resolve_btn.click(
        handle_resolve,
        inputs=[ticket_no_resolve, resolution_note, dept_selector],
        outputs=[resolution_status, dept_hist, dept_stats, dept_open_tickets, dept_closed_tickets, ticket_no_resolve, resolution_note]
    )

    chart_btn.click(generate_charts, None, [c1, c2, c3])
    export_btn.click(export_admin, None, file_out)

    def refresh_admin():
        return get_ticket_stats(), view_db()

    view_btn.click(refresh_admin, None, [stats_display, db_table])

app.launch(share=True, debug=True)

✅ Database initialized with email column


/tmp/ipython-input-3294537077.py:283: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  app = gr.Blocks(css=custom_css)


📊 Emergency stats - Total: 0, Open: 0, Closed: 0
📊 Found 0 OPEN tickets for Emergency department
📊 Found 0 CLOSED tickets for Emergency department
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1531d88671646cc31d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2202, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1924, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1879, in validate_outputs
    rai

📧 Email sent to: rsvinit.1010@gmail.com
🌐 API called: 404


...

📊 Emergency stats - Total: 1, Open: 1, Closed: 0
📊 Found 1 OPEN tickets for Emergency department
📊 Found 0 CLOSED tickets for Emergency department


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2202, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1924, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1879, in validate_outputs
    rai

📧 Email sent to: rsvinit.1010@gmail.com
📊 Emergency stats - Total: 1, Open: 0, Closed: 1
📊 Found 0 OPEN tickets for Emergency department
📊 Found 1 CLOSED tickets for Emergency department


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2202, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1924, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1879, in validate_outputs
    rai

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1531d88671646cc31d.gradio.live
